In [3]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from pinecone import Pinecone, ServerlessSpec
import json
import os
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()

pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

In [5]:
pc.create_index(
    name="professor-rate",
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

In [6]:
import json

data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. Alice Johnson',
  'subject': 'Mathematics',
  'stars': 4,
  'review': 'Dr. Johnson is an excellent professor who explains concepts clearly and is always available for extra help.'},
 {'professor': 'Prof. Bob Smith',
  'subject': 'Computer Science',
  'stars': 5,
  'review': 'Prof. Smith is very knowledgeable and makes complex topics easy to understand. Highly recommended!'},
 {'professor': 'Dr. Carol Williams',
  'subject': 'Biology',
  'stars': 3,
  'review': 'Dr. Williams is good, but the lectures can be a bit dry at times. The material is still well covered.'},
 {'professor': 'Dr. David Brown',
  'subject': 'Chemistry',
  'stars': 4,
  'review': 'Dr. Brown is engaging and makes chemistry interesting. Labs are well-organized and informative.'},
 {'professor': 'Prof. Emily Davis',
  'subject': 'Physics',
  'stars': 2,
  'review': "Prof. Davis's lectures are hard to follow. Needs to work on clarity and organization."},
 {'professor': 'Dr. Frank Miller',
  'subject':

In [8]:
hf_embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2")

c:\Users\moren\Downloads\project\prof-rate-ai\venv\lib\site-packages\langchain_core\_api\deprecation.py:151: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\moren\Downloads\project\prof-rate-ai\venv\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\moren\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

c:\Users\moren\Downloads\project\prof-rate-ai\venv\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
# Initialize an empty list to store processed data
processed_data = []

# Iterate through each review in the data
for review in data["reviews"]:
    # Combine relevant fields into a single string for embedding
    combined_text = f"{review['review']} {review['subject']} {review['stars']}"

    # Generate embedding for the combined text using Hugging Face's embeddings model
    embedding = hf_embeddings.embed_query(combined_text)

    # Append to processed data
    processed_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"]
        }
    })

In [11]:
print(processed_data)

[{'values': [0.004651263356208801, -0.0691385418176651, -0.005132345948368311, 0.009909357875585556, -0.08577517420053482, -0.0063455794006586075, -0.03406906872987747, 0.02001297101378441, -0.038158807903528214, 0.025485819205641747, -0.07621141523122787, 0.081802137196064, -0.07035507261753082, 0.08009804785251617, -0.012500499375164509, 0.059176892042160034, -0.03792952001094818, 0.034516382962465286, -0.045763492584228516, -0.10410282760858536, 0.08569756895303726, -0.05253142490983009, 0.021698173135519028, -0.04760492965579033, 0.03770144283771515, -0.028135614469647408, 0.04079246520996094, -0.05070922523736954, 0.08572965115308762, -0.009373496286571026, -0.07551074773073196, 0.02599632926285267, -0.018765956163406372, -0.009928219020366669, -0.05196728929877281, 0.028864679858088493, -0.024598760530352592, 0.1108960509300232, 0.03854846581816673, 0.09079793840646744, -0.0481354184448719, 0.02090042270720005, 0.06759747862815857, -0.03178659826517105, 0.04188578948378563, -0.03

In [12]:
index = pc.Index('professor-rate')
index.upsert(
  vectors=processed_data,
  namespace="ns1"
)

{'upserted_count': 21}

In [13]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 21}},
 'total_vector_count': 21}